In [1]:
import torch
masked_state_dict = torch.load('masked_gpt2.pt')
N=100

In [2]:
from transformer_lens.HookedTransformer import HookedTransformer
gpt2_masked = HookedTransformer.from_pretrained(is_masked=True, model_name='gpt2')

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [3]:
from transformer_lens.ioi_dataset import IOIDataset

ioi_dataset = IOIDataset(prompt_type="ABBA", N=N, nb_templates=1)
train_data = ioi_dataset.toks.long()


In [4]:
def logit_diff_from_ioi_dataset(
    logits: torch.Tensor, tokens: torch.Tensor, mean=False,
):
    assert tokens.shape == (
        N,
        16,
    ), tokens.shape  # TODO check this is not breaking things...
    assert len(logits.shape) == 3, logits.shape

    io_labels = tokens[:, 2]
    s_labels = tokens[:, 4]

    io_logits = logits[torch.arange(N), -2, io_labels]
    s_logits = logits[torch.arange(N), -2, s_labels]

    logit_diff = io_logits - s_logits
    if mean:
        return logit_diff.mean()
    else:
        return logit_diff


In [5]:
gpt2_masked.load_state_dict(masked_state_dict)
logit_diff_from_ioi_dataset(gpt2_masked(train_data), train_data, mean=True)

tensor(19.8824, device='cuda:0', grad_fn=<MeanBackward0>)

In [6]:
import numpy as np
mask_dict = np.load('mask_dict.npy', allow_pickle=True).item()
mask_dict["0.0.q"]

0.0

In [7]:
from transformer_lens import utils

def make_forward_hooks(nodes_to_mask):
    forward_hooks = []
    for layer in range(12):
        for head in range(12):
            for qkv in ["q", "k", "v"]:
                mask_value = nodes_to_mask[f"{layer}.{head}.{qkv}"]
                def head_ablation_hook(value, hook):
                    print(f"Shape of the value tensor: {value.shape}")
                    value[:, :, layer, :] *= mask_value
                    return value

                a_hook = (utils.get_act_name(qkv, int(head)), head_ablation_hook)
                forward_hooks.append(a_hook)
    return forward_hooks

forward_hooks = make_forward_hooks(mask_dict)


In [8]:
new_gpt2 = HookedTransformer.from_pretrained(is_masked=False, model_name='gpt2')
logit_diff_from_ioi_dataset(new_gpt2(train_data), train_data, mean=True)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


tensor(3.9698, device='cuda:0', grad_fn=<MeanBackward0>)

In [9]:
logit_diff_from_ioi_dataset(new_gpt2.run_with_hooks(train_data, return_type="logits", fwd_hooks=forward_hooks), train_data, mean=True)

Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torch.Size([100, 16, 12, 64])
Shape of the value tensor: torc

tensor(2.7861, device='cuda:0', grad_fn=<MeanBackward0>)